In [120]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import syft as sy
import copy
import numpy as np
import time
import binarytree

import importlib
importlib.import_module('FLDataset')
from FLDataset import load_dataset, getActualImgs
from utils import averageModels, normalModels

In [121]:
class Arguments():
    def __init__(self):
        self.images = 60000
        self.clients = 20
        self.rounds = 10
        self.epochs = 5
        self.local_batches = 64
        self.lr = 0.01
        self.C = 1.0
        self.drop_rate = 0.0
        self.mu = 0.5
        self.momentum = 0.6
        self.dampening = 1.0
        self.torch_seed = 0
        self.log_interval = 100
        self.iid = 'iid'
        self.split_size = int(self.images / self.clients)
        self.samples = self.split_size / self.images 
        self.use_cuda = False
        self.save_model = False

args = Arguments()

use_cuda = args.use_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [122]:
writer = open('prox.csv', 'a')

In [123]:
hook = sy.TorchHook(torch)
clients = []

for i in range(args.clients):
    clients.append({'hook': sy.VirtualWorker(hook, id="client{}".format(i+1))})

In [124]:
# # Download MNIST manually using 'wget' then uncompress the file
# !wget www.di.ens.fr/~lelarge/MNIST.tar.gz
# !tar -zxvf MNIST.tar.gz

In [125]:
global_train, global_test, train_group, test_group = load_dataset(args.clients, args.iid)

In [126]:
for inx, client in enumerate(clients):
    trainset_ind_list = list(train_group[inx])
    client['trainset'] = getActualImgs(global_train, trainset_ind_list, args.local_batches)
    client['testset'] = getActualImgs(global_test, list(test_group[inx]), args.local_batches)
#     client['samples'] = len(trainset_ind_list) / args.images
    client['samples'] = len(trainset_ind_list) / (len(trainset_ind_list) * args.clients)

In [127]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
global_test_dataset = datasets.MNIST('./', train=False, download=True, transform=transform)
global_test_loader = DataLoader(global_test_dataset, batch_size=args.local_batches, shuffle=True)

In [128]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [129]:
def ClientUpdate(args, device, client):
    client['model'].train()
#     client['model'].send(client['hook'])
    
    for epoch in range(1, args.epochs + 1):
        for batch_idx, (data, target) in enumerate(client['trainset']):
#             data = data.send(client['hook'])
#             target = target.send(client['hook'])
            
            data, target = data.to(device), target.to(device)
            client['optim'].zero_grad()
            output = client['model'](data)
            loss = F.nll_loss(output, target)
            loss.backward()
            client['optim'].step()
#             client['optim'].step(global_model)
    
            
            if batch_idx % args.log_interval == 0:
#                 loss = loss.get() 
                print('Model {} Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    client['hook'].id,
                    epoch, batch_idx * args.local_batches, len(client['trainset']) * args.local_batches, 
                    100. * batch_idx / len(client['trainset']), loss))
                
#     client['model'].get() 

In [130]:
def test(args, model, device, test_loader, name):
    model.eval()   
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss for {} model: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        name, test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    writer.write('{},'.format(100. * correct / len(test_loader.dataset)))

In [131]:
class FedProxOptim(optim.Optimizer):
    def __init__(self, params, lr=args.lr, mu=args.mu, momentum=args.momentum, dampening=args.dampening):
        defaults = dict(lr=lr, mu=mu, momentum=momentum, dampening=dampening)
        super(FedProxOptim, self).__init__(params, defaults)
    
    def step(self, global_model=None, closure = None):
        loss = None
        if closure is not None:
            loss = closure()
        for group in self.param_groups:
            lr, mu, momentum, dampening = group['lr'], group['mu'], group['momentum'], group['dampening']
            for p in zip(group['params'], list(global_model.parameters())):
                if p[0].grad is None:
                    continue
                d_p = p[0].grad.data # local model grads
                if momentum != 0:
                    param_state = self.state[p[0]]
                    if 'momentum_buffer' not in param_state:
                        buf = param_state['momentum_buffer'] = d_p.clone()
                    else:
                        buf = param_state['momentum_buffer']
                        buf.mul_(momentum).add_(1 - dampening, d_p)
                p[0].data.sub_(group['lr'], (d_p + mu * (p[0].data.clone() - p[1].data.clone())))
                
        return loss

In [132]:
def Aggregate(all_models, sample_client):
    all_dicts = {}
    for client in all_models:
        all_dicts[client] = all_models[client].state_dict()
    output_dict = copy.deepcopy(all_dicts[sample_client])
    for key in all_dicts[sample_client]:
        for val in all_dicts:
            if val == sample_client:
                continue
            output_dict[key] += all_dicts[val][key]
#         output_dict[key] /= float(len(all_dicts))
        output_dict[key] /= float(len(all_models))
    output_model = Net().to(device)
    output_model.load_state_dict(output_dict)
    return output_model

In [133]:
def BtreeAvg(root):
    global global_model
    temp_model = Aggregate({'root':root.value[1], 'global':global_model}, 'global')
    
    if (root.left == None and root.right == None): 
        return temp_model
    left = None
    right = None
    if root.left:
        left = BtreeAvg(root.left)
    if root.right:
        right = BtreeAvg(root.right)

    if left and right:
        return Aggregate({'root':root.value[1], 'left':left, 'right':right}, 'left')
    if left:
        return Aggregate({'root':root.value[1], 'left':left}, 'left')
    if right:
        return Aggregate({'root':root.value[1],'right':right}, 'right')
 

In [134]:
def BtreeAvgSecond(root, final):
    temp_model = Aggregate({'root':root.value[1], 'global':final}, 'global')
    
    if (root.left == None and root.right == None): 
        return temp_model
    left = None
    right = None
    if root.left:
        left = BtreeAvgSecond(root.left, temp_model)
    if root.right:
        right = BtreeAvgSecond(root.right, temp_model)

    if left and right:
        return Aggregate({'root':root.value[1], 'left':left, 'right':right}, 'left')
    if left:
        return Aggregate({'root':root.value[1], 'left':left}, 'left')
    if right:
        return Aggregate({'root':root.value[1],'right':right}, 'right')
 

In [135]:
torch.manual_seed(args.torch_seed)
global_model = Net()
# torch.manual_seed(args.torch_seed)
# normal_avg = Net()

for client in clients:
    torch.manual_seed(args.torch_seed)
    client['model'] = Net().to(device)
    client['optim'] = optim.SGD(client['model'].parameters(), lr=args.lr, momentum=args.momentum)
#     client['optim'] = optim.Adam(client['model'].parameters(), lr=args.lr)
#     client['optim'] = FedProxOptim(client['model'].parameters(), lr=args.lr, mu=args.mu, momentum=args.momentum, dampening=args.dampening)

tree_List = []
for inds, i in enumerate(clients):
    temp = [None] * 3
    temp[0] = i
    temp[1] = clients[inds]['model']
    temp[2] = clients[inds]['hook'].id
    tree_List.append(temp)

root = None
root = binarytree.insertLevelOrder(tree_List, root, 0, len(tree_List))
# root.printTree()

start_time = time.time()

for fed_round in range(args.rounds):
    
#     uncomment if you want a randome fraction for C every round
#     args.C = float(format(np.random.random(), '.1f'))
    
    # number of selected clients
    m = int(max(args.C * args.clients, 1))

    # Selected devices
    np.random.seed(fed_round)
    selected_clients_inds = np.random.choice(range(len(clients)), m, replace=False)
    selected_clients = [clients[i] for i in selected_clients_inds]
    
    # Active devices
    np.random.seed(fed_round)
    active_clients_inds = np.random.choice(selected_clients_inds, int((1-args.drop_rate) * m), replace=False)
    active_clients = [clients[i] for i in active_clients_inds]
    print(len(active_clients))
    
    # Training 
    for client in active_clients:
        ClientUpdate(args, device, client)
        
#     global_model = averageModels(global_model, active_clients)
#     test(args, global_model, device, global_test_loader, 'Global')
#     normal_avg = normalModels(normal_avg, active_clients)
#     test(args, normal_avg, device, global_test_loader, 'Global')
    
    
    # Begin the Btree
#     final_model = BtreeAvg(root)
    final_model = BtreeAvgSecond(root, global_model)
#     glo_model = copy.deepcopy(final_model)
    global_model.load_state_dict(final_model.state_dict())
#     test(args, global_model, device, global_test_loader, 'Global')
#     break
    
#     # Testing 
#     for client in active_clients:
#         test(args, client['model'], device, client['testset'], client['hook'].id)
    
#     # Averaging 
#     global_model = averageModels(global_model, active_clients)
    
    # Testing the average model
    test(args, global_model, device, global_test_loader, 'Global')
            
    # Share the global model with the clients
    for client in clients:
        client['model'].load_state_dict(global_model.state_dict())

writer.write('\n')
end_time = time.time() 
hours, rem = divmod(end_time-start_time, 3600)
minutes, seconds = divmod(rem, 60)
writer.write('{:0>2}:{:0>2}:{:05.2f}'.format(int(hours),int(minutes),seconds))
writer.write('\n\n')
writer.close()

if (args.save_model):
    torch.save(global_model.state_dict(), "FedAvg.pt")

20
Model client16 Train Epoch: 1 [0/3008 (0%)]	Loss: 2.317060
Model client16 Train Epoch: 2 [0/3008 (0%)]	Loss: 1.073454
Model client16 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.438347
Model client16 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.381759
Model client16 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.428643
Model client2 Train Epoch: 1 [0/3008 (0%)]	Loss: 2.300936
Model client2 Train Epoch: 2 [0/3008 (0%)]	Loss: 1.126643
Model client2 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.489269
Model client2 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.346984
Model client2 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.102700
Model client13 Train Epoch: 1 [0/3008 (0%)]	Loss: 2.317959
Model client13 Train Epoch: 2 [0/3008 (0%)]	Loss: 1.190133
Model client13 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.573687
Model client13 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.440502
Model client13 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.318959
Model client5 Train Epoch: 1 [0/3008 (0%)]	Loss: 2.315424
Model client5 Train Epoch: 2 [0/3008 (0%)]	Loss: 1.252286
M

Model client13 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.135724
Model client18 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.259950
Model client18 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.292756
Model client18 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.155723
Model client18 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.124694
Model client18 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.083712
Model client17 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.291406
Model client17 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.195229
Model client17 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.203227
Model client17 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.123091
Model client17 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.061555
Model client19 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.246606
Model client19 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.190214
Model client19 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.309447
Model client19 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.121907
Model client19 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.065730
Model client4 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.21631

Model client4 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.100813
Model client4 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.117985
Model client9 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.207916
Model client9 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.077368
Model client9 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.184203
Model client9 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.072874
Model client9 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.037987
Model client3 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.080683
Model client3 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.096388
Model client3 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.111473
Model client3 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.161006
Model client3 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.054630
Model client7 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.104175
Model client7 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.137133
Model client7 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.162743
Model client7 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.048277
Model client7 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.091234
Model client2 

Model client1 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.045323
Model client1 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.030820
Model client1 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.046671
Model client1 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.016936
Model client3 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.139597
Model client3 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.027819
Model client3 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.040706
Model client3 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.032281
Model client3 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.095159
Model client14 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.065285
Model client14 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.029214
Model client14 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.034214
Model client14 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.085326
Model client14 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.082439
Model client20 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.029063
Model client20 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.036591
Model client20 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.137553
Model 

Model client13 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.044752
Model client13 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.056480
Model client13 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.082914
Model client13 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.035184
Model client13 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.058522
Model client14 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.046820
Model client14 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.021728
Model client14 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.084893
Model client14 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.047359
Model client14 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.023336
Model client19 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.045900
Model client19 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.061499
Model client19 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.035619
Model client19 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.049020
Model client19 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.032120
Model client3 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.023681
Model client3 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.220481

Model client12 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.031951
Model client13 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.005614
Model client13 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.082716
Model client13 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.055606
Model client13 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.033041
Model client13 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.024949

Test set: Average loss for Global model: 0.0569, Accuracy: 9823/10000 (98%)

20
Model client18 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.033909
Model client18 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.027457
Model client18 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.024698
Model client18 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.060601
Model client18 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.013697
Model client4 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.013034
Model client4 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.058862
Model client4 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.072326
Model client4 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.019337
Model client4 Train Epoch: 5 [0/3008 (

Model client15 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.006213
Model client15 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.115076
Model client15 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.021530
Model client7 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.004212
Model client7 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.096366
Model client7 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.073966
Model client7 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.023352
Model client7 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.064112
Model client20 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.111052
Model client20 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.015956
Model client20 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.088598
Model client20 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.035998
Model client20 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.026870
Model client12 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.103458
Model client12 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.086818
Model client12 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.019498
Model client12 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.022966
Mo

Model client15 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.096983
Model client15 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.053218
Model client15 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.010005
Model client15 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.014871
Model client1 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.007994
Model client1 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.011517
Model client1 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.006765
Model client1 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.008153
Model client1 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.015196
Model client2 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.034327
Model client2 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.076923
Model client2 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.022986
Model client2 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.012745
Model client2 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.005349
Model client18 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.037776
Model client18 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.119831
Model client18 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.034315
Model c

In [136]:
clients[0]['optim']

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.01
    momentum: 0.6
    nesterov: False
    weight_decay: 0
)